In [ ]:
    # loading the dataset
    import pandas as pd
    import numpy as np
    df = pd.read_csv('dataset/ks-projects-201612.csv',  sep = ',')    
    # abcd

In [52]:
print(df.head())

     uniqueId                                              name   \
0  1000002330                    The Songs of Adelaide & Abullah   
1  1000004038                                     Where is Hank?   
2  1000007540  ToshiCapital Rekordz Needs Help to Complete Album   
3  1000011046  Community Film Project: The Art of Neighborhoo...   
4  1000014025                               Monarch Espresso Bar   

        category  main_category  currency       deadline   goal   \
0          Poetry     Publishing       GBP  10/9/15 11:36   1000   
1  Narrative Film   Film & Video       USD   2/26/13 0:20  45000   
2           Music          Music       USD   4/16/12 4:24   5000   
3    Film & Video   Film & Video       USD   8/29/15 1:00  19500   
4     Restaurants           Food       USD   4/1/16 13:38  50000   

       launched  pledged       state  backers  country  usd pledged   \
0  8/11/15 12:12        0      failed        0       GB            0   
1   1/12/13 0:20      220      failed 

In [53]:
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323750 entries, 0 to 323749
Data columns (total 17 columns):
uniqueId          323750 non-null int64
name              323748 non-null object
category          323745 non-null object
main_category     323750 non-null object
currency          323750 non-null object
deadline          323750 non-null object
goal              323750 non-null object
launched          323750 non-null object
pledged           323750 non-null object
state             323750 non-null object
backers           323750 non-null object
country           323750 non-null object
usd pledged       319960 non-null object
Unnamed: 13       625 non-null object
Unnamed: 14       12 non-null object
Unnamed: 15       4 non-null object
Unnamed: 16       1 non-null float64
dtypes: float64(1), int64(1), object(15)
memory usage: 42.0+ MB
None


In [48]:
print (df.dtypes)


uniqueId            int64
name               object
category           object
main_category      object
currency           object
deadline           object
goal               object
launched           object
pledged            object
state              object
backers            object
country            object
usd pledged        object
Unnamed: 13        object
Unnamed: 14        object
Unnamed: 15        object
Unnamed: 16       float64
dtype: object


In [49]:
df.describe()
        

,uniqueId,Unnamed: 16
count,3.237500e+05,1.0
mean,1.074943e+09,25.0
std,6.193352e+08,NaN
min,5.971000e+03,25.0
25%,5.375802e+08,25.0
50%,1.075818e+09,25.0
75%,1.611070e+09,25.0
max,2.147476e+09,25.0


Column Metadata

column name                         Description                Type
uniqueID                            Id of project              String
name         Title of the project
String
category
Kickstarter sub category
String
main_category
Kickstarter main category
String
currency
Intial currency of the project
String
deadline
Project deadline
DateTime
goal
Project goal
Numeric
launched
Project launch date
DateTime
pledged
Amount pledge in inital currency
Numeric
state
Project current state
String
backers
Number of backers
Numeric
country
Country of project creator
String
usd pledged
Kickstarter conversion of pledged amount in USD
Numeric
